# Парсер сообщений из дискуссий группы Я донор в ВК

Импорт библиотек

In [ ]:
import pandas as pd
import numpy as np

import os
import copy

import requests
from bs4 import BeautifulSoup

from tqdm.notebook import tqdm
from time import sleep

from google.colab import files
from google.colab import drive

Для Google Colab подключение Google диска

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/projects/donor-search-files/парсер

/content/drive/MyDrive/projects/donor-search-files/парсер


## Парсинг списка обсуждений

In [ ]:
url = 'https://webcache.googleusercontent.com/search?q=cache:Iwt4HPGNHIMJ:https://vk.com/board5307143+&cd=2&hl=ru&ct=clnk&gl=ru'

Загрузим страницу

In [ ]:
r = requests.get(url)
r.text

'<!DOCTYPE html><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><base href="https://vk.com/board5307143"><style>body{margin-left:0;margin-right:0;margin-top:0}#bN015htcoyT__google-cache-hdr{background:#f8f9fa;font:13px arial,sans-serif;text-align:left;color:#202124;border:0;margin:0;border-bottom:1px solid #dadce0;line-height:16px;padding:16px 28px 24px 28px}#bN015htcoyT__google-cache-hdr *{display:inline;font:inherit;text-align:inherit;color:inherit;line-height:inherit;background:none;border:0;margin:0;padding:0;letter-spacing:0}#bN015htcoyT__google-cache-hdr a{text-decoration:none;color:#1967d2}#bN015htcoyT__google-cache-hdr a:hover{text-decoration:underline}#bN015htcoyT__google-cache-hdr a:visited{color:#4b11a8}#bN015htcoyT__google-cache-hdr div{display:block;margin-top:4px}#bN015htcoyT__google-cache-hdr b{font-weight:bold;display:inline-block;direction:ltr}</style><div id="bN015htcoyT__google-cache-hdr"><div><span>Это версия страницы <a href="https://vk.com/board

Превратим в читабельный вид

In [ ]:
soup = BeautifulSoup(r.text, 'lxml')
soup

<!DOCTYPE html>
<html><head><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><base href="https://vk.com/board5307143"/><style>body{margin-left:0;margin-right:0;margin-top:0}#bN015htcoyT__google-cache-hdr{background:#f8f9fa;font:13px arial,sans-serif;text-align:left;color:#202124;border:0;margin:0;border-bottom:1px solid #dadce0;line-height:16px;padding:16px 28px 24px 28px}#bN015htcoyT__google-cache-hdr *{display:inline;font:inherit;text-align:inherit;color:inherit;line-height:inherit;background:none;border:0;margin:0;padding:0;letter-spacing:0}#bN015htcoyT__google-cache-hdr a{text-decoration:none;color:#1967d2}#bN015htcoyT__google-cache-hdr a:hover{text-decoration:underline}#bN015htcoyT__google-cache-hdr a:visited{color:#4b11a8}#bN015htcoyT__google-cache-hdr div{display:block;margin-top:4px}#bN015htcoyT__google-cache-hdr b{font-weight:bold;display:inline-block;direction:ltr}</style></head><body><div id="bN015htcoyT__google-cache-hdr"><div><span>Это версия страницы <a

Запишем все обсуждения (топики) в переменную

In [ ]:
topics_info = soup.findAll('div', class_='blst_info')

Кол-во топиков совпадает с тем, что на самом деле

In [ ]:
len(topics_info)

27

Так можем получить кусок ссылки на топик

In [ ]:
topics_info[2].find('a', class_='blst_title').get('href')

'/topic-5307143_25692597'

Так получим названия топика

In [ ]:
topics_info[2].find('a', class_='blst_title').text

'"Почетный донор России". Вопросы и ответы.'

Так выглядит третий по счету топик

In [ ]:
topics_info[2]

<div class="blst_info">
<div class="blst_title_wrap">
<a class="blst_title" href="/topic-5307143_25692597" onclick="return nav.go(this, event)">"Почетный донор России". Вопросы и ответы.</a>
</div>
<div class="blst_other">
      11<span class="num_delim"> </span>429 сообщений<span class="divider">|</span>Стр. <a class="blst_page" href="/topic-5307143_25692597?offset=0">1</a> <a class="blst_page" href="/topic-5307143_25692597?offset=20">2</a> <a class="blst_page" href="/topic-5307143_25692597?offset=40">3</a> .. <a class="blst_page" href="/topic-5307143_25692597?offset=11380">570</a> <a class="blst_page" href="/topic-5307143_25692597?offset=11400">571</a> <a class="blst_page" href="/topic-5307143_25692597?offset=11420">572</a>
</div>
</div>

Так можно получить номер последней страницы обсуждения

In [ ]:
int(topics_info[2].findAll('a', class_='blst_page')[-1].text)

572

Создадим переменную, где сохраним информацию по каждой дискуссии

In [ ]:
topics = []

Перебираем обсуждения, нужную информацию добавляем в список

In [ ]:
for topic in topics_info:
    topic_url = 'https://vk.com' + topic.find('a', class_='blst_title').get('href')
    topic_name = topic.find('a', class_='blst_title').text
    last_page_number = int(topic.findAll('a', class_='blst_page')[-1].text)
    
    topics.append([topic_url, topic_name, last_page_number])

Для наглядности эти данные добавим в датафрейм

In [ ]:
topics_infotable = pd.DataFrame(topics, columns=['topic_url', 'topic_name', 'last_page_number'])
display(topics_infotable)

,topic_url,topic_name,last_page_number
0,https://vk.com/topic-5307143_22305652,Общие вопросы,1722
1,https://vk.com/topic-5307143_33165077,Почетный донор Москвы: звание и льготы,91
2,https://vk.com/topic-5307143_25692597,"""Почетный донор России"". Вопросы и ответы.",572
3,https://vk.com/topic-5307143_29147436,Доноры Санкт-Петербурга,85
4,https://vk.com/topic-5307143_31892654,Почетный донор Москвы (обсуждения),130
5,https://vk.com/topic-5307143_28816758,Какую информацию о донорстве крови и ее компон...,13
6,https://vk.com/topic-5307143_25556165,▌► ЕСЛИ ВАМ НУЖЕН ДОНОР...,42
7,https://vk.com/topic-5307143_25197449,Что у вас в ОПК подается к чаю?,62
8,https://vk.com/topic-5307143_26279245,"Для тех,кто может предоставить помощь.Пишите а...",44
9,https://vk.com/topic-5307143_26021880,Обращение к опытным донорам,34


Сохраним эту табличку в формате csv

In [ ]:
topics_infotable.to_csv('topics_infotable.csv', index=False, encoding='utf-8')

## Парсинг сообщений

Теперь, когда есть список обсуждений и обозначено кол-во страниц для каждого из них, можем заняться парсингом сообщений

In [ ]:
#создаем переменную url
#http://webcache.googleusercontent.com/search?q=cache: первая часть ссылки, после вставляем нужный нам адрес
url = "http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_28346460?offset=240"

#Загрузим страницу целиком
r = requests.get(url)

#с помощью beautifulsoup приведем к читабельному виду
soup = BeautifulSoup(r.text, 'lxml')

In [ ]:
soup

<!DOCTYPE html>
<html lang="en">
<head><meta charset="utf-8"/>
<meta content="initial-scale=1, minimum-scale=1, width=device-width" name="viewport"/>
<title>Error 404 (Not Found)!!1</title>
<style>
    *{margin:0;padding:0}html,code{font:15px/22px arial,sans-serif}html{background:#fff;color:#222;padding:15px}body{margin:7% auto 0;max-width:390px;min-height:180px;padding:30px 0 15px}* > body{background:url(//www.google.com/images/errors/robot.png) 100% 5px no-repeat;padding-right:205px}p{margin:11px 0 22px;overflow:hidden}ins{color:#777;text-decoration:none}a img{border:0}@media screen and (max-width:772px){body{background:none;margin-top:0;max-width:none;padding-right:0}}#logo{background:url(//www.google.com/images/branding/googlelogo/1x/googlelogo_color_150x54dp.png) no-repeat;margin-left:-5px}@media only screen and (min-resolution:192dpi){#logo{background:url(//www.google.com/images/branding/googlelogo/2x/googlelogo_color_150x54dp.png) no-repeat 0% 0%/100% 100%;-moz-border-image:url(

In [ ]:
soup.find('div', class_="bp_post clear_fix")

Похоже, что < div class="bp_post clear_fix "> то, что нам нужно,
содержит инфо об авторе и само сообщение.
a class="bp_author" - id и имя автора поста;
a class="bp_date" - дата и время поста

### Запускаем процесс

In [ ]:
CACHE_URL = 'http://webcache.googleusercontent.com/search?q=cache:' #ссылка, где хранятся кэшированные страницы
OFFSET_URL = '?offset=' #параметр offset для навигации по страницам обсуждения
data_messages = [] #Здесь храним всю информацию из сообщений
recaptcha_url_list = [] #url, где возникала капча (нужно проверить)
empty_pages = [] #ссылки, по которым не удалось загрузить результаты. Могут быть и 404 и капчи
parsed_pages = [] #где храним сохраненные страницы

In [ ]:
def recaptcha_warning(page_to_check, url):
    """
    Функция для фиксирования капчи.
    Принимает html страницу и url.
    печатает оповещение и url
    """
    if page_to_check.select("#recaptcha"):
        print('КАПЧА!', url)
        recaptcha_url_list.append(url)
        sleep(180)
        page = BeautifulSoup(requests.get(url).text, 'lxml')

загрузим списки уже обработанных сообщений

In [ ]:
empty_pages = np.load('empty_pages_list.npy').tolist()
parsed_pages = np.load('parsed_pages_list.npy').tolist()
data_messages = pd.read_csv('data_messages_0.csv', index_col=0).loc[:, '1':].values.tolist() #здесь начинаем с первого, чтобы не грузить индексы из CSV

In [ ]:
for i in tqdm(range(len(topics))):
    #Создаем url первой страницы
    url = CACHE_URL + topics[i][0]

    #вычисляем последний номер параметра offset
    offset_last_number = topics[i][2] * 20
    
    for offset_number in tqdm(range(20, offset_last_number+1, 20)):
        try:
            if url in empty_pages:
                print(url, 'есть в списке незагруженных')
                url = CACHE_URL + topics[i][0] + OFFSET_URL + str(offset_number)
                continue
            elif url in parsed_pages:
                print(url, 'есть в списке загруженных')
                url = CACHE_URL + topics[i][0] + OFFSET_URL + str(offset_number)
                continue
            #загружаем страницу
            page = BeautifulSoup(requests.get(url).text, 'lxml')
            #пауза, чтобы не перегружать сервер и попытка избежать капчу
            sleep(np.random.randint(30, 90))
            #проверка на капчу
            recaptcha_warning(page, url)
            #подгружаем список постов
            bp_posts = page.findAll("div", class_="bp_post clear_fix ")
            #если рез-тов нет, то добавим такую страницу в empty_pages
            if not bp_posts:
                empty_pages.append(url)
                print(url, 'не загрузилась')
            else:
                parsed_pages.append(url)
                print(url, 'ЗАГРУЖЕНА')
                
            for bp_post in bp_posts:
                author_link = "https://vk.com" + bp_post.find("a", class_="bp_author").get("href")
                author_name = bp_post.find("a", class_="bp_author").text
                bp_post_time = bp_post.find("a", class_="bp_date").text
                bp_post_text_orig = bp_post.find("div", class_="bp_text")
                like_button_count = bp_post.find('div', class_='like_button_count').text
                data_messages.append([topics[i][1], #Название темы
                                     url, #ссылка на страницу с сообщением
                                     author_link, #ссылка на автора
                                     author_name, #имя автора
                                     bp_post_time, #время сообщения
                                     bp_post_text_orig, #текст сообщения в оригинале с тегами и т.п.
                                     like_button_count]) #кол-во лайков под постом
            url = CACHE_URL + topics[i][0] + OFFSET_URL + str(offset_number)
            
        except Exception as e:
            print(e, 'in', url)
        #чтобы не потерять ничего будем сохранять данные
        try:
            pd.DataFrame(data_messages).to_csv('./data_messages_'+str(i)+'.csv')
            np.save('empty_pages_list.npy', empty_pages)
            np.save('parsed_pages_list.npy', parsed_pages)
            #files.download('./data_messages_'+str(i)+'.csv')
        except Exception as e:
            print(e, 'ошибка при создании или загрузке файла')

  0%|          | 0/27 [00:00<?, ?it/s]

  0%|          | 0/1722 [00:00<?, ?it/s]

http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_22305652 есть в списке загруженных
http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_22305652?offset=20 есть в списке незагруженных
http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_22305652?offset=40 есть в списке незагруженных
http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_22305652?offset=60 есть в списке незагруженных
http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_22305652?offset=80 есть в списке загруженных
http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_22305652?offset=100 есть в списке незагруженных
http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_22305652?offset=120 есть в списке незагруженных
http://webcache.googleusercontent.com/search?q=cache:https://vk.com/topic-5307143_22305652?offset=140 есть в 